In [321]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.decomposition as skd 
#have a look at the documentation for skd here 
#"https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html"

In [322]:
def normalize(data):
    return (data - data.mean())/data.std()

In [323]:
# def pca(data, n): #perform PCA on data to reduce dimensionality (change the n_components / features to see how it affects the results)
#     normalize(data)    
#     pca = skd.PCA(n_components = n)
#     pca.fit(data)
#     return pca.transform(data)

In [324]:
# def pca(data, n): #perform PCA on data to reduce dimensionality (change the n_components / features to see how it affects the results)
#     transposed = data.transpose() #to calculate covariance matrix, find transpose
#     intermediate = np.dot(transposed,data)
#     cov_matrix = (1/(2000-1)) * intermediate #covariance matrix

#     #print(cov_matrix.shape)

#     eigenvalues,eigenvectors = np.linalg.eig(cov_matrix) #eigenvalues and eigenvectors for covariance matrix
#     #print(len(eigenvalues))
#     # print((eigenvectors.shape))

#     variation_matrix=np.zeros((2352,1)) # corresponds to each eigenvalue to see which contributes most to the data
#     summation = np.sum(eigenvalues)
#     for i in range(2352):
#         variation_matrix[i][0]=eigenvalues[i]/summation

#     reduced_eigenvectors = eigenvectors[:,:n] #2352x21
#     print(reduced_eigenvectors.shape)
#     return np.dot(data,reduced_eigenvectors)

In [325]:
def pca(data, n): #perform PCA on data to reduce dimensionality (change the n_components / features to see how it affects the results)
    cov_matrix = np.cov(data.T) #covariance matrix
    eigenvalues,eigenvectors = np.linalg.eig(cov_matrix) #eigenvalues and eigenvectors for covariance matrix
    variation_matrix = []

    for i in eigenvalues:
        variation_matrix.append(i/np.sum(eigenvalues)*100) #get the percentage of variation for each eigenvalue
    
    cumulative_variation = np.cumsum(variation_matrix) #cumulative variation -> get components that contribute most to the data
    print(cumulative_variation) #-> we can use 10 componets to explain most of the features
    #We only use the above code if we want to see the percentage of variation for each eigenvalue

    projected_data = np.dot(data,eigenvectors[:,:n]) #projected data
    return projected_data

In [326]:
data = np.loadtxt("inputs.txt")
labels = np.loadtxt("labels.txt")

In [327]:
m, n = data.shape #m is the number of data-points /samples, n is the number of features
n = 10 #testing with 5 features out of 2352

In [345]:
#code to shuffle 2 arrays, and keep corresponding elements
randomize = np.arange(len(labels)) 
np.random.shuffle(randomize) #creates a randomized sequence to be used as an index for the two arrays to shuffle them (https://www.delftstack.com/howto/numpy/python-numpy-shuffle-two-arrays/)

data = data[randomize]
labels = labels[randomize]

data = pca(data, n)

#This is to avoid dimension issues with np.dot 
#Also...I took small samples of the date to test the ann
training_data = data[:10].T
Y_training = labels[:10]
X_training = training_data[0:n]

validation_data = data[10:16].T #note, includes start index, excludes end index
Y_validation = labels[10:16]
X_validation = training_data[0:n]

testing_data = data[16:24].T
Y_testing = labels[16:24]
X_testing = training_data[0:n]

print(data.shape)

[ 21.52663766+0.j  35.52385919+0.j  48.52958238+0.j  52.28408258+0.j
  56.64006322+0.j  62.69159041+0.j  70.3856342 +0.j  78.93483253+0.j
  88.75218839+0.j 100.        +0.j]
(2000, 10)


In [329]:
def sigmoid(Z):
    return 1/(1 + np.exp(-Z))

In [330]:
def sigmoid_prime(Z):
    return Z * (1 - Z)

In [331]:
def softmax(Z):
    return np.exp(Z)/np.sum(np.exp(Z), axis=0)

In [332]:
def rand_params(): #generate a random set of weights and biases for the neural network between -1 and 1
    w1= np.random.rand(5, n) - 1 # n is the number of features
    b1 = np.random.rand(5, 1) - 1
    w2 = np.random.rand(5, 5) - 1
    b2 = np.random.rand(5, 1) - 1
    w3 = np.random.rand(10, 5) - 1
    b3 = np.random.rand(10, 1) - 1
    return w1, b1, w2, b2, w3, b3

In [333]:
def forward_prop(X, w1, b1, w2, b2, w3, b3): #forward propagation
    Z1 = np.dot(w1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(w2, A1) + b2
    A2 = sigmoid(Z2)
    Z3 = np.dot(w3, A2) + b3
    A3 = softmax(Z3)
    return A1, A2, A3

In [334]:
def one_hot_encode(labels): #encode labels as one-hot vectors
    labels = labels.astype(int)
    encoded_labels = np.zeros((labels.size, 10))
    for i in range(labels.size):
        encoded_labels[i][labels[i]] = 1
    return encoded_labels.T

In [335]:
print(one_hot_encode(Y_training))
print(Y_training)

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[2. 4. 4. 6. 0. 2. 1. 7. 2. 7.]


In [336]:
def regularize(w1, w2, w3): #regularization (lambda = 0.99)
    w1 = w1 * 0.99
    w2 = w2 * 0.99
    w3 = w3 * 0.99

In [337]:
#Parameters:
#w1, b1, w2, b2, w3, b3: weights and biases
#X: training data
#Y: training labels
#A1, A2, A3: activation functions

#Returns:
#dw1, db1, dw2, db2, dw3, db3: deltas for weights and biases
def backprop(X, Y, A1, A2, A3, w1, b1, w2, b2, w3, b3): #backpropagation
    Y = one_hot_encode(Y)
    dZ3 = A3 - Y
    dW3 = np.dot(dZ3, A2.T)
    db3 = np.sum(dZ3, axis=1, keepdims=True)
    dA2 = np.dot(w3.T, dZ3)
    dZ2 = dA2 * sigmoid_prime(A2)
    dW2 = np.dot(dZ2, A1.T)
    db2 = np.sum(dZ2, axis=1, keepdims=True)
    dA1 = np.dot(w2.T, dZ2)
    dZ1 = dA1 * sigmoid_prime(A1)
    dW1 = np.dot(dZ1, X.T)
    db1 = np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2, dW3, db3

In [338]:
#Parameters:
#dW1, db1, dW2, db2, dW3, db3: deltas for weights and biases

#Returns:   
#w1, b1, w2, b2, w3, b3: updated weights and biases
def update_params(w1, b1, w2, b2, w3, b3, dW1, db1, dW2, db2, dW3, db3, learning_rate): #update parameters
    w1 = w1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    w2 = w2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    w3 = w3 - learning_rate * dW3
    b3 = b3 - learning_rate * db3
    regularize(w1, w2, w3) #regularize weights
    return w1, b1, w2, b2, w3, b3

In [339]:
def get_predictions(A3): 
    return np.argmax(A3,0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

In [340]:
def gradient_descent(X, Y, epochs, learning_rate): #gradient descent -> learn weights and biases
    w1, b1, w2, b2, w3, b3 = rand_params()
    for i in range(epochs):
        A1, A2, A3 = forward_prop(X, w1, b1, w2, b2, w3, b3)
        dW1, db1, dW2, db2, dW3, db3 = backprop(X, Y, A1, A2, A3, w1, b1, w2, b2, w3, b3)
        w1, b1, w2, b2, w3, b3 = update_params(w1, b1, w2, b2, w3, b3, dW1, db1, dW2, db2, dW3, db3, learning_rate)
    
    print("Epochs: ",epochs,"|","Training Accuracy: ", get_accuracy(get_predictions(A3), Y))
    print("Predicted: ",get_predictions(A3),"\n", "Actual: ",Y)
    return w1, b1, w2, b2, w3, b3

In [348]:
#learnt set of weights and biases (this is basically what we are submitting)
w1, b1, w2, b2, w3, b3 = gradient_descent(X_training, Y_training, 2000, 0.01)

Epochs:  2000 | Training Accuracy:  1.0
Predicted:  [8 8 6 8 8 1 9 9 4 4] 
 Actual:  [8. 8. 6. 8. 8. 1. 9. 9. 4. 4.]
